In [2]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError
from urllib3.exceptions import MaxRetryError

In [3]:
def visit(url):
    try:
        response = requests.get(url)
        print("EL status code: ", response.status_code)
        response.raise_for_status()  # arroja si la solicitud no se concluye correctamente
        if(response.status_code == 200):
            #response.request.header muestra la forma en como hago la solicitud
            html = BeautifulSoup(response.text, 'html.parser')
            return html
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except MaxRetryError as Max_err:
        print(f'HTTP error MaxEntry ocurred: {Max_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    return None

In [159]:
url = 'https://www.latamairlines.com/ec/es/ofertas-vuelos?dataFlight=%7B%22tripTypeSelected%22%3A%7B%22label%22%3A%22Ida%22%2C%22value%22%3A%22OW%22%7D%2C%22cabinSelected%22%3A%7B%22label%22%3A%22Economy%22%2C%22value%22%3A%22Economy%22%7D%2C%22passengerSelected%22%3A%7B%22adultQuantity%22%3A1%2C%22infantQuantity%22%3A0%2C%22childrenQuantity%22%3A0%7D%2C%22originSelected%22%3A%7B%22airportName%22%3A%22Mariscal%20Sucre%20Int.%22%2C%22countryName%22%3A%22Ecuador%22%2C%22cityName%22%3A%22Quito%22%2C%22airportIataCode%22%3A%22UIO%22%2C%22countryAlpha2%22%3A%22EC%22%7D%2C%22destinationSelected%22%3A%7B%22airportName%22%3A%22Ministro%20Pistarini%20Int.%22%2C%22countryName%22%3A%22Argentina%22%2C%22cityName%22%3A%22Buenos%20Aires%22%2C%22airportIataCode%22%3A%22EZE%22%2C%22countryAlpha2%22%3A%22AR%22%7D%2C%22dateGoSelected%22%3A%222020-10-31T17%3A00%3A00.000Z%22%2C%22dateReturnSelected%22%3Anull%2C%22redemption%22%3Afalse%7D&sort=RECOMMENDED'
htm = visit(url)

EL status code:  200


In [177]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as Ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException


In [ ]:
driver = webdriver.Edge(executable_path = 'D:/edgeDriver/msedgedriver')
#driver abre otro navegador que podremos ejecutar desde el codigo
"""hacer que el navegador cargue la pagina web"""
driver.get(url)

In [ ]:
#agregar demora dinamica para darle tiempo al navegador
delay = 10 #maximo de tiempo a esperar
try:
    vuelo = WebDriverWait(driver,delay).until(Ec.presence_of_element_located((By.XPATH,'//li[@class="sc-jGFFOr jWBHtm"]')))
    print('la pagina termino de cargar')
    scrapperLatam()
except TimeoutException:
    print('La pagina tardo demasiado en cargar')

In [123]:
def getBasicFlyInfo(vuelo):
    
    dataFly = vuelo.find_elements_by_xpath('.//div[@class="sc-dPNhBE fEZYFJ flight-information"]')
    #el punto para indicar que solo busque desde donde me encuentro hacia abajo
   #con las //le digo que la informacion no es un hijo directo de este vuelo
    dicc={}
    i=0
    for data in dataFly:
        hora = data.find_element_by_xpath('.//span[@class="sc-izvnbC hveHla"]').text
        
       
        if i:
            lugarLlegada = data.find_element_by_xpath('//span[@class="sc-cbMPqi lfmmmk"]').text
            dicc["Llegada"]=lugarLlegada
            dicc['HoraLlegada']=hora
            
        else: 
            lugarSalida = data.find_element_by_xpath('//span[@class="sc-cbMPqi liUWFk"]').text
            dicc["Salida"]=lugarSalida
            dicc['HoraSalida']= hora
        i+=1
            
            
    duracion = vuelo.find_element_by_xpath('.//span[@class="sc-gMcBNU kUEpCT"]').get_attribute('textContent')
    dicc["Duracion"]=duracion
    return dicc

    
    


In [193]:
def getInfoDepartureArrival(segmento):
    """method who add the data at in a dictionary for the scale per segment
    [departurePlace,arrivalPlace,timeDeparture,timeArrival,duration,timeWaitAirport,flyNumber,airplaneNumber]"""
    
    WebDriverWait(driver,10).until(Ec.presence_of_element_located((By.XPATH,'//div[@class="sc-imapFV jQvkv"]/span')))
    
    departureData = segmento.find_elements_by_xpath('.//div[@class="sc-jgVwMx guTIbo"]/div[@class="iataCode"]/span')
    arrivalData = segmento.find_elements_by_xpath('.//div[@class="sc-AUpyg dvLpUb"]/div[@class="iataCode"]/span')
    duration = segmento.find_elements_by_xpath('.//div[@class="sc-kWHCRG bZmbtE"]/span')[1].text
    timeWait = segmento.find_element_by_xpath('//div[@class="sc-imapFV jQvkv"]/span').text    
    flyNumber = segmento.find_element_by_xpath('.//div[@class="sc-dzOgQY cgKyuN airline-wrapper"]').text
    airplaneModel = segmento.find_element_by_xpath('.//span[@class="airplane-code"]').text
    print("DepartureData: ",departureData)
    dicc = {
        "Departure":departureData[0].text,
        "Arrival":arrivalData[0].text,
        "TimeDeparture":departureData[1].text,
        "TimeArrival":arrivalData[1].text,
        "Duration":duration,
        "TimeToWait":timeWait,
        "FlyNumber":flyNumber,
        "AirplaneModel":airplaneModel
    }
    return dicc
        
    
    

In [194]:
def getInfoConectionFlys(segmentos):
    l = []
    WebDriverWait(driver,10).until(Ec.presence_of_element_located((By.XPATH,'//article[@class="sc-hCbubC CNDJE"]')))
    for segmento in segmentos:
        l.append(getInfoDepartureArrival(segmento))
    #cerramos la miniventana sobre las escalas
    driver.find_element_by_xpath('//button[@class="MuiButtonBase-root MuiIconButton-root sc-jbKcbu icxyJV"]').click()
    return l

In [195]:
def obtenerTarifas(vuelo):
    print("Getting prices for the fligth")
    l=[]
    vuelo.click()
    tarifas = vuelo.find_elements_by_xpath('//ol[@class ="sc-iBfVdv hZEqyB"]/div')
    for tar in tarifas:
        plan = tar.find_element_by_xpath('.//span[@class = "sc-hPeUyl lkBXZN"]').text
        precio = tar.find_element_by_xpath('.//span[@class = "sc-jWxkHr bZLAFU"]').text 
        currency = tar.find_element_by_xpath('.//span[@class = "sc-dPPMrM cnZPMA"]').text
        dicc = {plan:{'Precio':precio,'Moneda':currency}}
        l.append(dicc)
    
    vuelo.find_element_by_xpath('.//button[@class= "sc-imDfJI jLHJeB"]').click()
    return l
    
    
    
   

In [196]:
obtenerTarifas(vuelo)

Getting prices for the fligth


ElementClickInterceptedException: Message: element click intercepted: Element <li class="sc-jGFFOr jWBHtm">...</li> is not clickable at point (328, 433). Other element would receive the click: <div class="MuiDialog-container MuiDialog-scrollPaper" role="none presentation" type="aside" tabindex="-1" style="transform: none; transition: transform 225ms cubic-bezier(0, 0, 0.2, 1) 0ms;">...</div>
  (Session info: MicrosoftEdge=85.0.564.51)


In [ ]:
def scrapperLatam():
    vuelos = driver.find_elements_by_xpath('//li[@class="sc-jGFFOr jWBHtm"]')
    print(len(vuelos))
    i=0
    for vuelo in vuelos:
        print("Starting Scrapper for fly: {}".format(i))
        print(getBasicFlyInfo(vuelo))
        botonEscalas = vuelo.find_element_by_xpath('.//a[@class="sc-frDJqD dFTGrD"]')
        if botonEscalas!=None:
            botonEscalas.click()
            WebDriverWait(driver,10).until(Ec.presence_of_element_located((By.XPATH,'//section[@class="sc-kMBllD jaBnqa"]')))
            segmentos = vuelo.find_elements_by_xpath('//section[@class="sc-kMBllD jaBnqa"]')
            print("el len de segmentos: ",len(segmentos))
            prueba = vuelo.find_elements_by_xpath('//section[@class="sc-hCbubC CNDJE"]/section[@class="sc-kMBllD jaBnqa"]')
            print("el len de pruebas: ",len(prueba))
            escalas = len(segmentos)-1
            print("Scales you have to do: {}".format(escalas))
            print("Scales Information")
            print(getInfoConectionFlys(segmentos))
        print(obtenerTarifas(vuelo))
    
    driver.close()
            
        
    